## **Name:  Muhammad Bilal**
## **Roll No:  BSCS201919**

# Active Learning

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import uuid
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Problem Statement

The business problem we are going to solve is associated to demographic modelling. Given some vital statistics associated to the users on our platform, we want to identify the gender associated to the user. We have a large amount of data but little to no labelled data. The goal is to build a model to identify the gender of a user.

## Loading Data

In [3]:
infile = open('/content/drive/MyDrive/FYP/Training_data.txt', 'r', encoding='utf-8-sig')
train = infile.readlines()
infile.close()

train_words = []
train_tags = []
for i in train:
    data = i.split("\n")[0]
    data = data.split("\t")
    train_words.append(data[0])
    train_tags.append(data[1])


In [6]:
sentences = []
pos_tags = []
index = 0
for i in range(len(train_words)):
  if train_words[i]=='۔':
    sentences.append(train_words[index:i+1])
    pos_tags.append(train_tags[index:i+1])
    index = i+1

In [7]:
len(sentences)

42211

In [ ]:
test_words = []
infile = open('/content/drive/MyDrive/FYP/data_lstm.txt', 'r', encoding='utf-8-sig')
test = infile.readlines()
infile.close()
for i in test:
    data = i.split("), ")
    data = [w.replace('(','') for w in data]
    data = [w.replace('\'','') for w in data]
    data = [w.replace('\n','') for w in data]
    data = [w.split(', ') for w in data]
    for item in data:
        test_words.append(item[0])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
train_tokens = [tokenizer(i) for i in train_words]

In [ ]:
tokens = [np.array(token['input_ids']) for token in train_tokens]

In [ ]:
from keras.utils import pad_sequences
MAX_LENGTH = 10

tokens = pad_sequences(tokens, maxlen=MAX_LENGTH, padding='post')

In [ ]:
def get_train_tags_y(train_tags, tag2index):
    train_tags_y = []
    for tags in train_tags:
      train_tags_y.append(tag2index[tags])
    return train_tags_y

In [ ]:
tag2index = {t: i + 1 for i, t in enumerate(list(train_tags))}
tag2index['-PAD-'] = 0
index2tag = {v:k for k,v in tag2index.items()}

In [ ]:
tags = get_train_tags_y(train_tags, tag2index)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    tokens, 
    tags,
    test_size = 0.4
)

## Train Model

In [ ]:
# Random Forest Classifier
clf = RandomForestClassifier()

# train the model
clf.fit(x_train, y_train)

RandomForestClassifier()

## Model Accuracy

In [ ]:
def clf_eval(clf, x_test, y_test):
    '''
    This function will evaluate a sk-learn multi-class classification model based on its
    x_test and y_test values
    
    params:
        clf (Model) : The model you wish to evaluate the performance of
        x_test (Array) : Result of the train test split
        y_test (Array) : Result of the train test split
    
    returns:
        This function will return the following evaluation metrics:
            - Accuracy Score
            - Matthews Correlation Coefficient
            - Classification Report
            - Confusion Matrix
    
    example:
        clf_eval(
            clf,
            x_test,
            y_test
        )
    '''
    y_pred = clf.predict(x_test)
    y_true = y_test
    
    y_pred = clf.predict(x_test)
    test_acc = accuracy_score(y_test, y_pred)
    print("Testing Accuracy : ", test_acc)
    
    print("MCC Score : ", matthews_corrcoef(y_true, y_pred))
    
    print("Classification Report : ")
    print(classification_report(y_test, clf.predict(x_test)))
    
    
clf_eval(
    clf,
    x_test,
    y_test
)

Testing Accuracy :  0.903370963922002
MCC Score :  0.8878079413314879
Classification Report : 
              precision    recall  f1-score   support

      114161       0.00      0.00      0.00         1
      114253       0.00      0.00      0.00         1
      163030       0.74      0.15      0.25        93
      163502       0.67      0.85      0.75       627
      173529       0.00      0.00      0.00         1
      173880       0.00      0.00      0.00         1
      173883       0.00      0.00      0.00         1
      179375       0.00      0.00      0.00         1
      183127       0.98      0.95      0.97       111
      184937       0.50      0.23      0.32        13
      185753       1.00      0.82      0.90        22
      186537       0.91      0.38      0.54        79
      189067       0.98      0.96      0.97       395
      189311       0.98      1.00      0.99       260
      189359       0.96      0.99      0.97       878
      189426       0.00      0.00      0

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Predict

In [ ]:
pred_df = pd.DataFrame(
        {
            'words' : test_words
        })

In [ ]:
test_tokens = [tokenizer(i) for i in pred_df['words']]

In [ ]:
test_tokens = [np.array(token['input_ids']) for token in test_tokens]

In [ ]:
from keras.utils import pad_sequences
test_tokens = pad_sequences(test_tokens, maxlen=MAX_LENGTH, padding='post')

In [ ]:
pred_proba= clf.predict_proba(test_tokens)
pred = clf.predict(test_tokens)

In [ ]:
pred_proba

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.46584146, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.99983437]])

In [ ]:
pred_df['pred_proba'] = [i for i in pred_proba]
pred_df['pred'] = [i for i in pred]

## Get Annotated & To Annoate Predictions

Predictions with low predicted probabilities will be annotated manually by the user, while predictions which have high predicted probability will be assumed to be a good prediction generated by the model. This will allow us to increase our labelled data and retrain a new model based on the best performing results of the previous model.

In [ ]:
# low predictions with threshold <= 0.6
low_th = 0.6

# high predictions with threshold >= 0.9
high_th = 0.9

def check_preds(pred_proba_list, low_th = low_th, high_th = high_th):
    '''
    This function will check the dictionary associated to the prediction probabilities
    generated by the model. It will return either `annotate`, `annotated` or `neither`
    as a result.
    
    params:
        pred_dct (Dictionary) : The keys are the classes, values are the predicted proba
        low_th (Integer) : The low prediction proba threshold
        high_th (Integer): The high prediction proba threshold
        
    returns:
        This function will return the following:
            `annotate` : if the maximum value in the pred_dct is less than or equal to
                         the low_th
            `annotated` : if the maximum value in the pred_dct is greater than or equal to
                          the high_th
            'neither' : If it does not fall in the other two ranges
            
    example:
        pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)
    '''
    
    max_val = max(pred_proba_list)
    if max_val <= low_th:
        return 'annotate'
    elif max_val >= high_th:
        return 'annotated'
    else:
        return 'neither'

In [ ]:
pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)

In [ ]:
pred_df['annotate_decision'].value_counts()

annotated    692503
neither      202031
annotate     105422
Name: annotate_decision, dtype: int64

In [ ]:
pred_df.head()

,words,pred_proba,pred,annotate_decision
0,’,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",190004,annotated
1,میرے,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",189311,annotated
2,بھائی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",190001,annotated
3,کا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",189982,annotated
4,ای,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",190001,annotated


In [ ]:
new_tags = []
new_words = []
untagged_words = []
for index, row in pred_df.iterrows():
    if row['annotate_decision']=='annotated':
        new_tags.append(index2tag[row['pred']])
        new_words.append(row['words'])
    else:
      untagged_words.append(row['words'])

## Retrain Model

In [ ]:
updated_words = train_words+new_words
updated_tags = train_tags+new_tags

In [ ]:
updated_words = [tokenizer(i) for i in updated_words]

In [ ]:
updated_tokens = [np.array(token['input_ids']) for token in updated_words]

In [ ]:
updated_tokens = pad_sequences(updated_tokens, maxlen=MAX_LENGTH, padding='post')

In [ ]:
updated_tags_ids = get_train_tags_y(updated_tags, tag2index)

In [ ]:
# train test split
x = updated_tokens
y = updated_tags_ids

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.3
)

In [ ]:
# DTC classifier
clf = RandomForestClassifier()

# train the model
clf.fit(x_train, y_train)

RandomForestClassifier()

## Model Accuracy

In [ ]:
clf_eval(
    clf,
    x_test,
    y_test
)

Testing Accuracy :  0.980496947157269
MCC Score :  0.9765588164176973
Classification Report : 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      114161       0.00      0.00      0.00         1
      163030       0.88      0.41      0.56        87
      163502       0.62      0.88      0.72       465
      173883       0.00      0.00      0.00         1
      183127       0.99      0.98      0.99       237
      184937       0.38      0.60      0.46        10
      185753       0.97      0.97      0.97        96
      186537       1.00      0.89      0.94       231
      189067       1.00      0.99      1.00      1672
      189311       1.00      1.00      1.00      1618
      189359       1.00      1.00      1.00      4728
      189426       0.00      0.00      0.00        74
      189459       0.99      1.00      1.00       120
      189516       1.00      1.00      1.00      1117
      189584       0.99      0.98      0.99       367
      189739       0.74      0.93      0.82       218
      189751       1.00      0.99      0.99      1157
      189830       0.62    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pred_proba= clf.predict_proba(test_tokens)
pred = clf.predict(test_tokens)

In [ ]:
pred_df['pred_proba'] = [i for i in pred_proba]
pred_df['pred'] = [i for i in pred]

## Get Annotated & To Annoate Predictions

Predictions with low predicted probabilities will be annotated manually by the user, while predictions which have high predicted probability will be assumed to be a good prediction generated by the model. This will allow us to increase our labelled data and retrain a new model based on the best performing results of the previous model.

In [ ]:
pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)

In [ ]:
pred_df['annotate_decision'].value_counts()

annotated    733365
neither      166336
annotate     100255
Name: annotate_decision, dtype: int64

In [ ]:
new_tags = []
new_words = []
untagged_words = []
for index, row in pred_df.iterrows():
    if row['annotate_decision']=='annotated':
        new_tags.append(index2tag[row['pred']])
        new_words.append(row['words'])
    else:
      untagged_words.append(row['words'])

In [ ]:
with open("/content/drive/MyDrive/FYP/Training_data.txt", "a") as file:
  for word, tag in zip(new_words, new_tags):
    file.write(f"{word}\t{tag}\n")
